## LU- и LUP-разложение
<b>Олохтонова Мария Сергеевна
    НПМбд-01-19</b>

<b>LU-разложение</b>  — представление матрицы A в виде произведения двух матриц,<b>A = L * U</b> , где L — нижняя треугольная матрица, а U —верхняя треугольная матрица.



Для работы с алгоритмами необходимо импортировать библиотеки numpy и scipy (scipy для проверки работы алгоритмов).

In [1]:
import numpy as np
import scipy.linalg

In [7]:
def LU(A):
    n = A.shape[0]
    L = np.eye(n)
    U = np.zeros((n, n))

    for x in range(n * n):
        i = x // n
        j = x % n
        if i <= j:
            U[i, j] = A[i, j] - np.dot(L[i, :i], U[:i, j])
        else:
            L[i, j] = (A[i, j] - np.dot(L[i, :j], U[:j, j])) / U[j, j]

    return L, U

В LU-разложении мы делим на диагональный элемент матрицы А, поэтому он не должен быть равен 0.

In [3]:
def test_lu(A):
    L, U = LU(A)

    print('L = \n', L, '\n')
    print('U = \n', U, '\n')
    print('L * U = \n', L @ U, '\n')
mtx = np.array([[1, 2, 3], 
                [4, 5, 6], 
                [7, 8, 9]], dtype=float)
test_lu(mtx)
# mtx = np.array([[1, 1, 0], [0, 6, 1], [8, 0, 0]], dtype=float)
print('С нулями на диагонали:', '\n')
diag_nul_mtx = np.array([[0, 0, 1],
                         [0, 1, 0],
                         [1, 0, 0]], dtype=float)
test_lu(diag_nul_mtx)

L = 
 [[1. 0. 0.]
 [4. 1. 0.]
 [7. 2. 1.]] 

U = 
 [[ 1.  2.  3.]
 [ 0. -3. -6.]
 [ 0.  0.  0.]] 

L * U = 
 [[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]] 

С нулями на диагонали: 

L = 
 [[ 1.  0.  0.]
 [nan  1.  0.]
 [inf nan  1.]] 

U = 
 [[ 0.  0.  1.]
 [ 0. nan nan]
 [ 0.  0. nan]] 

L * U = 
 [[ 0. nan nan]
 [nan nan nan]
 [nan nan nan]] 



/tmp/ipykernel_91296/2008434355.py:12: RuntimeWarning: invalid value encountered in double_scalars
  L[i, j] = (A[i, j] - np.dot(L[i, :j], U[:j, j])) / U[j, j]
/tmp/ipykernel_91296/2008434355.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  L[i, j] = (A[i, j] - np.dot(L[i, :j], U[:j, j])) / U[j, j]
/tmp/ipykernel_91296/2294609705.py:6: RuntimeWarning: invalid value encountered in matmul
  print('L * U = \n', L @ U, '\n')


Такое разложение может быть использовано для решения семейства систем линейных уравнений с различными векторами b в правой части:
<b>Ax = b</b>. Где A - невырожденная матрица коэффициентов системы.
Тогда CЛАУ представляется в виде: <b>LUx = b</b> 

<b>LUx = b</b> 
Эта система может быть решена в два шага. На первом шаге решается система <b>Ly=b</b>. Поскольку L — нижняя треугольная матрица, эта система решается непосредственно прямой подстановкой:

На втором шаге решается система <b>Ux=y</b>. <b></b> Поскольку U — верхняя треугольная матрица, эта система решается непосредственно обратной подстановкой.


In [4]:
def LU_solver(A, b):
    L, U = LU(A)
    n = A.shape[0]

    # L * y = b
    y = np.array([b[0]/L[0, 0]])
    for i in range(1, n):
        yi = (b[i] - np.dot(L[i, :i], y))/L[i, i]
        y = np.append(y, yi)

    # U * x = y
    # Используем такой же алгоритм, но переворачиваем матрицу
    # и вектор значений (np.flip)
    U = np.flip(U)
    y = np.flip(y)
    x = np.array([y[0] / U[0, 0]])
    for i in range(1, n):
        xi = (y[i] - np.dot(U[i, :i], x)) / U[i, i]
        x = np.append(x, xi)

    return np.flip(x)

In [8]:
mtx = np.array([[1, 2, 3], 
                [4, 5, 6], 
                [7, 8, 10]], dtype=float)
b = np.array([1, 1, 1])
print(LU_solver(mtx,b))

[-1.  1.  0.]


Проверяем, сравнивая с numpy

In [9]:
np.linalg.solve(mtx, b)

array([-1.00000000e+00,  1.00000000e+00,  3.80647894e-16])

<b>LUP-разложение</b>
В LU-разложении мы делим на диагональный элемент матрицы А, но может быть и так, что он равен нулю. Тогда мы можем в изначальной матрице переставлять строки, чтобы на диагонали был не 0, и запоминать перестановки в матрицу перестановок P.
Ищем матрицу перестановок:

In [10]:
def partial_pivot(A):
    n = A.shape[0]
    permutation = []
    already = set()
    indices = set([_ for _ in range(n)])

    for i in range(n):
        pivot = None
        pivot_value = None

        column = A[:, i]  # колонка номер i
        good_indices = indices - already  # только индексы элементов, которых нет в already
        with_indices = np.array(list(map(lambda i: (i, column[i]), good_indices)), dtype=int)  # пары (индекс, значение)
        fast_pivot = max(with_indices, key=lambda e: np.abs(e[1]))[0]  # максимум из этих пар по модулю значения

        # print(pivot, fast_pivot)

        permutation.append(fast_pivot)
        already.add(fast_pivot)

    permutation_matrix = np.zeros(A.shape)
    for i, item in enumerate(permutation):
        permutation_matrix[i, item] = 1

    return permutation_matrix



PLU должен работать и с матрицами, у которых 0 на главной диагонали.

In [11]:
def test_plu(A):
    P = partial_pivot(A)
    A_p = P @ A
    L, U = LU(A_p)

    print('P = \n', P)
    print('L = \n', L)
    print('U = \n', U)

    print('P * A = \n', P @ A)
    print('L * U = \n', L @ U)

    # P * A = L * U           | * P^(-1)
    # A = P^(-1) * L * U
test_plu(mtx)

P = 
 [[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
L = 
 [[1.         0.         0.        ]
 [0.57142857 1.         0.        ]
 [0.14285714 2.         1.        ]]
U = 
 [[ 7.          8.         10.        ]
 [ 0.          0.42857143  0.28571429]
 [ 0.          0.          1.        ]]
P * A = 
 [[ 7.  8. 10.]
 [ 4.  5.  6.]
 [ 1.  2.  3.]]
L * U = 
 [[ 7.  8. 10.]
 [ 4.  5.  6.]
 [ 1.  2.  3.]]


In [13]:
def PLU_solver(A, b):
    P = partial_pivot(A)
    b = P.dot(b)
    A = P.dot(A)
    return LU_solver(A, b)


In [14]:
print(PLU_solver(mtx,b))

[-1.0000000e+00  1.0000000e+00  4.4408921e-16]
